In [ ]:
import geopandas as gpd

In [31]:
# Ref: https://gf0604-procesamientodatosgeograficos.github.io/2022-i/datos/sinac/areas-silvestres-protegidas.geojson
asp = gpd.read_file("asp.geojson")

In [33]:
# Ensure geometry is in EPSG:4326 for degrees
asp = asp.to_crs("EPSG:4326")

# Calculate centroids for all geometries
asp["centroid"] = asp.geometry.centroid

# # Extract latitude and longitude from centroids
asp["longitude"] = asp.centroid.x
asp["latitude"] = asp.centroid.y


/var/folders/dh/s7j1y_hn3jj5n5jrdhz6_9j80000gn/T/ipykernel_29032/2564850709.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  asp["centroid"] = asp.geometry.centroid
/var/folders/dh/s7j1y_hn3jj5n5jrdhz6_9j80000gn/T/ipykernel_29032/2564850709.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  asp["longitude"] = asp.centroid.x
/var/folders/dh/s7j1y_hn3jj5n5jrdhz6_9j80000gn/T/ipykernel_29032/2564850709.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  asp["latitude"] = asp.centroid.y


In [34]:
asp.to_csv("asp_lat_long.csv")